# Capítulo 7 - DQN


Você pode rodar este notebook no Colab ou localmente. Para abrir diretamente no Colab, basta clicar no link abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap07/cap07-main.ipynb) 

Para rodar localmente, primeiro, baixe todo o repositório do github: https://github.com/pablo-sampaio/rl_facil.

## 1. Configurações Iniciais

### Configurações Dependentes do Sistema

Rode a célula abaixo, mesmo sem estar no Colab.

In [1]:
import sys
from IPython.display import clear_output

if 'google.colab' in sys.modules:
    !pip install gym
    !pip install gym[box2d]
    !pip install gym[atari]
    !pip install opencv-python
    !pip install autorom[accept-rom-license]
    !pip install tensorboard

    # para salvar videos
    !apt-get install -y xvfb x11-utils
    !pip install pyvirtualdisplay==0.2.*
    !apt-get install ffmpeg

    from pyvirtualdisplay import Display
    #global display
    display = Display(visible=False, size=(1400, 900))
    _ = display.start()

    !git clone https://github.com/pablo-sampaio/rl_facil
    !mv /content/rl_facil/cap07/* /content/


In [2]:
# Make video
# Set up fake display; otherwise rendering will fail
#import os
#os.system("Xvfb :1 -screen 0 0124x768x24 &")
#os.environ['DISPLAY'] = ':1'

### Configurações para Exibir Video

In [3]:
# ideias adaptadas de : https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google
from base64 import b64encode
from IPython.display import HTML
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  html_code = f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'
  return HTML(html_code)

## 2. Imports e Definições Usadas pelo DQN

Código adaptado do código explicado no livro de M. Lapan, cap. 6.

### Imports

In [6]:
import dqn_models
from atari_wrappers import *

import argparse
import time
from datetime import datetime
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.optim as optim

#from tensorboardX import SummaryWriter
#from torch.utils.tensorboard import SummaryWriter

%load_ext tensorboard

### Classes Auxiliares

In [ ]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])


class DQNExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, s1, a, r, done, s2):
        experience = Experience(s1, a, r, done, s2)
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)


### Funções Auxiliares

In [ ]:
# Faz uma escolha epsilon-greedy
def choose_action(qnet, env, state, epsilon, device):
    done_reward = None
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state_a = np.array([state], copy=False)
        state_v = torch.tensor(state_a, dtype=torch.float32).to(device)
        q_vals_v = qnet(state_v)
        _, act_v = torch.max(q_vals_v, dim=1)
        action = int(act_v.item())

    return action

In [ ]:
def test_Qpolicy(env, Qpolicy, epsilon=0.0, num_episodes=5, render=False, videorec=None):
    episodes_returns = []
    total_steps = 0
    num_actions = env.action_space.n
    #device = Qpolicy.get_device()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for i in range(num_episodes):
        obs = env.reset()
        if render:
            env.render()
        if videorec is not None:
            videorec.capture_frame()
        done = False
        steps = 0
        episodes_returns.append(0.0)
        while not done:
            action = choose_action(Qpolicy, env, obs, epsilon, device)
            obs, reward, done, _ = env.step(action)
            if render:
                env.render()
            if videorec is not None:
                videorec.capture_frame()
            total_steps += 1
            episodes_returns[-1] += reward
            steps += 1
        print(f"EPISODE {i+1}")
        print("- steps:", steps)
        print("- return:", episodes_returns[-1])
    mean_return = round(np.mean(episodes_returns), 1)
    print("RESULTADO FINAL: média (por episódio):", mean_return, end="")
    print(", episódios:", len(episodes_returns), end="")
    print(", total de passos:", total_steps)
    if videorec is not None:
        videorec.close()
    return mean_return, episodes_returns


### Função de Perda (*Loss Function*)

In [ ]:
def calc_loss(batch, net, tgt_net, gamma, device="cpu"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states, dtype=torch.float32).to(device)
    next_states_v = torch.tensor(next_states, dtype=torch.float32).to(device)
    actions_v = torch.tensor(actions, dtype=torch.int64).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.tensor(dones, dtype=torch.bool).to(device)

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * gamma + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

## 3 - Função Principal do DQN

Esta é a função que faz o aprendizado. (Porém, o DQN é uma solução que incluir também as ideias dos wrappers.)

In [ ]:
def DQN_TRAIN(env, env_name, qnet, qnet_lr, tgt_qnet, target_update_freq, gamma, replay_size, batch_size, epsilon_f, epsilon_decay_last_step, MEAN_REWARD_BOUND):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    qnet = qnet.to(device)
    tgt_qnet = tgt_qnet.to(device)

    writer = SummaryWriter(comment="-" + env_name)
    print(qnet)

    buffer = DQNExperienceBuffer(replay_size)
    epsilon = 1.0

    optimizer = optim.Adam(qnet.parameters(), lr=qnet_lr)
    total_rewards = []
    frame_idx = 0
    ts_frame = 0
    ts = time.time()
    best_mean_reward = None
    start_time_str = datetime.now().strftime("%Y-%m-%d,%H-%M-%S")

    state = env.reset()
    total_reward = 0.0

    while True:
        frame_idx += 1
        epsilon = max(epsilon_f, 1.0 - frame_idx / epsilon_decay_last_step)

        action = choose_action(qnet, env, state, epsilon, device)

        # do step in the environment
        new_state, reward, is_done, _ = env.step(action)
        total_reward += reward

        buffer.append(state, action, reward, is_done, new_state)
        state = new_state
        
        if is_done:
            total_rewards.append(total_reward)
            state = env.reset()
            total_reward = 0.0
            if (time.time() - ts) == 0:
                speed = float("-inf")
            else:
                speed = (frame_idx - ts_frame) / (time.time() - ts)
            ts_frame = frame_idx
            ts = time.time()
            mean_reward = np.mean(total_rewards[-100:])
            print("%d: done %d games, mean reward %.3f, eps %.2f, speed %.2f f/s" % (
                frame_idx, len(total_rewards), mean_reward, epsilon,
                speed
            ))
            writer.add_scalar("epsilon", epsilon, frame_idx)
            writer.add_scalar("speed", speed, frame_idx)
            writer.add_scalar("reward_100", mean_reward, frame_idx)
            writer.add_scalar("reward", reward, frame_idx)
            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(qnet.state_dict(), env_name + "-" + start_time_str + "-best.dat")
                # IDEIA: salvar também a iteração, para poder retomar
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f -> %.3f, model saved" % (best_mean_reward, mean_reward))
                best_mean_reward = mean_reward
            if mean_reward > MEAN_REWARD_BOUND:
                print("Solved in %d frames!" % frame_idx)
                break

        if len(buffer) < replay_size:
            continue

        if frame_idx % target_update_freq == 0:
            tgt_qnet.load_state_dict(qnet.state_dict())

        optimizer.zero_grad()
        batch = buffer.sample(batch_size)
        loss_t = calc_loss(batch, qnet, tgt_qnet, gamma, device=device)
        loss_t.backward()
        optimizer.step()

    writer.close()


## 4 - Rodando em Ambientes Simples

In [ ]:
ENV_NAME = "MountainCar-v0"
REWARD_BOUND = -120
#ENV_NAME = "CartPole-v0"
#REWARD_BOUND = 200

GAMMA = 0.999
REPLAY_SIZE = 2000
BATCH_SIZE = 32
LEARNING_RATE = 0.001
SYNC_TARGET_FRAMES = 250

EPSILON_DECAY_LAST_FRAME = 100000
EPSILON_FINAL = 0.02

In [ ]:
env = gym.make(ENV_NAME)

qnet = dqn_models.MLP(env.observation_space.shape[0], [128,256], env.action_space.n)
qtarget = dqn_models.MLP(env.observation_space.shape[0], [128,256], env.action_space.n)

In [ ]:
# Iniciando a visualização antes da execução, para acompanhar
%tensorboard --logdir runs

In [ ]:
DQN_TRAIN(
    env = env, 
    env_name = ENV_NAME, 
    qnet = qnet,
    qnet_lr = LEARNING_RATE,
    tgt_qnet = qtarget, 
    target_update_freq = SYNC_TARGET_FRAMES,
    gamma = GAMMA, 
    replay_size = REPLAY_SIZE, 
    batch_size = BATCH_SIZE, 
    epsilon_f = EPSILON_FINAL, 
    epsilon_decay_last_step = EPSILON_DECAY_LAST_FRAME, 
    MEAN_REWARD_BOUND = REWARD_BOUND)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=3, bias=True)
  )
)
200: done 1 games, mean reward -200.000, eps 1.00, speed 17700.47 f/s
400: done 2 games, mean reward -200.000, eps 0.99, speed 15918.87 f/s
600: done 3 games, mean reward -200.000, eps 0.99, speed 22539.72 f/s
800: done 4 games, mean reward -200.000, eps 0.99, speed 17249.15 f/s
1000: done 5 games, mean reward -200.000, eps 0.99, speed 19348.21 f/s
1200: done 6 games, mean reward -200.000, eps 0.98, speed 15326.70 f/s
1400: done 7 games, mean reward -200.000, eps 0.98, speed 22668.85 f/s
1600: done 8 games, mean reward -200.000, eps 0.98, speed 16580.90 f/s
1800: done 9 games, mean reward -200.000, eps 0.98, speed 18375.12 f/s
2000: done 10 games, mean reward -200.000, eps 0.97, speed 17746.91 f/s
2200: done 11 games, mean reward -200.000, e

In [ ]:
# Para carregar de arquivo
#qnet.load_state_dict(torch.load("/content/MountainCar-v0-XXXXXX-best.dat", map_location=lambda storage, loc: storage))

# Faz alguns testes com o modelo e salva o vídeo em arquivo
video = VideoRecorder(env, "politica-treinada.mp4")
test_Qpolicy(env, qnet, 0.0, 20, render=False, videorec=None)

EPISODE 1
- steps: 89
- return: -89.0
EPISODE 2
- steps: 106
- return: -106.0
EPISODE 3
- steps: 106
- return: -106.0
EPISODE 4
- steps: 126
- return: -126.0
EPISODE 5
- steps: 87
- return: -87.0
EPISODE 6
- steps: 95
- return: -95.0
EPISODE 7
- steps: 105
- return: -105.0
EPISODE 8
- steps: 86
- return: -86.0
EPISODE 9
- steps: 126
- return: -126.0
EPISODE 10
- steps: 94
- return: -94.0
EPISODE 11
- steps: 107
- return: -107.0
EPISODE 12
- steps: 107
- return: -107.0
EPISODE 13
- steps: 87
- return: -87.0
EPISODE 14
- steps: 106
- return: -106.0
EPISODE 15
- steps: 91
- return: -91.0
EPISODE 16
- steps: 97
- return: -97.0
EPISODE 17
- steps: 98
- return: -98.0
EPISODE 18
- steps: 88
- return: -88.0
EPISODE 19
- steps: 105
- return: -105.0
EPISODE 20
- steps: 106
- return: -106.0
RESULTADO FINAL: média (por episódio): -100.6, episódios: 20, total de passos: 2012


(-100.6,
 [-89.0,
  -106.0,
  -106.0,
  -126.0,
  -87.0,
  -95.0,
  -105.0,
  -86.0,
  -126.0,
  -94.0,
  -107.0,
  -107.0,
  -87.0,
  -106.0,
  -91.0,
  -97.0,
  -98.0,
  -88.0,
  -105.0,
  -106.0])

In [ ]:
render_mp4("politica-treinada.mp4")

## 4. Rodando no Jogo Pong (Atari)

In [ ]:
--
ENV_NAME = "PongNoFrameskip-v4"
REWARD_BOUND = 15 #19.5

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 3e-4 #1e-4
SYNC_TARGET_FRAMES = 1000

EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_FINAL = 0.02

In [ ]:
env = gym.make(ENV_NAME)
env = MaxAndSkipEnv(env)
env = FireResetEnv(env)
env = ProcessFrame84(env)
env = ImageToPyTorch(env)
env = BufferWrapper(env, 4)

net = dqn_model.DQNNet(env.observation_space.shape, env.action_space.n)
tgt_net = dqn_model.DQNNet(env.observation_space.shape, env.action_space.n)

In [ ]:
DQN_TRAIN(
    env = env, 
    env_name = ENV_NAME, 
    qnet = qnet,
    qnet_lr = LEARNING_RATE,
    tgt_qnet = qtarget, 
    target_update_freq = SYNC_TARGET_FRAMES,
    gamma = GAMMA, 
    replay_size = REPLAY_SIZE, 
    batch_size = BATCH_SIZE, 
    epsilon_f = EPSILON_FINAL, 
    epsilon_decay_last_step = EPSILON_DECAY_LAST_FRAME, 
    MEAN_REWARD_BOUND = REWARD_BOUND)

In [ ]:
# Faz alguns testes com o modelo de forma DETERMINÍSTICA e salva o vídeo em arquivo
video = VideoRecorder(ENV, "politica-treinada.mp4")
test_policy(ENV, policy, True, 5, render=False, videorec=video)